# Census - Median Household Income (MHI)

#### This notebook contains code for gathering Census data tables and converting them to dataframes. 
#### Please use caution when altering this code.

In [1]:
import pandas as pd
from census import Census #<-- Python wrapper for census API
import requests
import os
import matplotlib.pyplot as plt
import matplotlib

# Census API Key
from config import api_key

# provide the api key and the year to establish a session
c = Census(api_key, year=2018)

# Set an option to allow up to 300 characters to print in each column
pd.set_option('max_colwidth', 300)

In [2]:
tables = c.acs5.tables()

# The tables variable contains a list of dicts, so we can convert directly to a dataframe
table_df = pd.DataFrame(tables)

##### The cell below gathering columns names from the Census Total Population table that only contain estimate and are int data types.
##### Those table names are added to a string variable and then pulled from the census data.
##### Dictionary key names are replaced withe more meaningful names.
##### Data is saved to a dataframe and exported as a csv file.

In [12]:
table_id = 'B19013'      

# Capture the variables URL from the table_df
url = table_df.loc[table_df['name']==table_id, 'variables'].values[0]

# Make the API call
response = requests.get(url).json()

# convert the response to a DataFrame
variables = pd.DataFrame(response['variables']).transpose()

print(f"Number of available variables: {len(variables)}")

table_info = variables[(variables['predicateType']=='int') & (variables['label'].str.contains("Estimate"))]   


table_columns = ""
for ind in table_info.index:       
    table_columns = table_columns + f"{ind},"
    
    
column_list = table_columns[:-1]

census_data = c.acs5.get(("NAME", column_list), 
                          {'for': 'county:*'})

# for i in range(len(table_info)) : 
#     for d in census_data:
#         d[table_info.iloc[i, 0]] = d.pop(table_info.index[i]) 
    

census_B19013_MHI_df = pd.DataFrame(census_data)
census_B19013_MHI_df.to_csv("census_B19013_MHI.csv", encoding="utf-8", index=False )
census_B19013_MHI_df

print(type(census_data))
print(census_data)

Number of available variables: 4
<class 'list'>
[{'NAME': 'Washington County, Mississippi', 'B19013_001E': 30834.0, 'state': '28', 'county': '151'}, {'NAME': 'Perry County, Mississippi', 'B19013_001E': 39007.0, 'state': '28', 'county': '111'}, {'NAME': 'Choctaw County, Mississippi', 'B19013_001E': 37203.0, 'state': '28', 'county': '019'}, {'NAME': 'Itawamba County, Mississippi', 'B19013_001E': 40510.0, 'state': '28', 'county': '057'}, {'NAME': 'Carroll County, Mississippi', 'B19013_001E': 43060.0, 'state': '28', 'county': '015'}, {'NAME': 'Grenada County, Mississippi', 'B19013_001E': 36877.0, 'state': '28', 'county': '043'}, {'NAME': 'Jefferson County, Mississippi', 'B19013_001E': 20188.0, 'state': '28', 'county': '063'}, {'NAME': 'Greene County, Mississippi', 'B19013_001E': 49447.0, 'state': '28', 'county': '041'}, {'NAME': 'Marshall County, Mississippi', 'B19013_001E': 41572.0, 'state': '28', 'county': '093'}, {'NAME': 'Quitman County, Mississippi', 'B19013_001E': 25510.0, 'state': '

### Census MHI Column Changes

In [5]:
census_B19013_MHI_df.dtypes

census_B19013_MHI_df['COUNTYFP'] = census_B19013_MHI_df['state'] + census_B19013_MHI_df['county']

census_MHI_sorted = census_B19013_MHI_df.sort_values(by=['COUNTYFP'])
census_MHI_sorted = census_MHI_sorted.reset_index(drop=True)

census_MHI_sorted['State Name'] = ''
census_MHI_sorted['County Name'] = ''


for index, row in census_MHI_sorted.iterrows():
    name_all = row['NAME']
    name_list = name_all.split(',')
    clean_state = name_list[1][1:]
    census_MHI_sorted.loc[index, 'State Name']= clean_state
    census_MHI_sorted.loc[index, 'County Name']= name_list[0]
    
census_MHI_sorted.head()

,NAME,state,county,Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars),COUNTYFP,State Name,County Name
0,"Autauga County, Alabama",01,001,58786.0,01001,Alabama,Autauga County
1,"Baldwin County, Alabama",01,003,55962.0,01003,Alabama,Baldwin County
2,"Barbour County, Alabama",01,005,34186.0,01005,Alabama,Barbour County
3,"Bibb County, Alabama",01,007,45340.0,01007,Alabama,Bibb County
4,"Blount County, Alabama",01,009,48695.0,01009,Alabama,Blount County
